# Proyecto de detección de fraudes de transacciones con Tarjetas de Crédito (TDC)

## Definición de problema y objetivo

About the Dataset

This is a simulated credit card transaction dataset containing legitimate and fraud transactions from the duration 1st Jan 2019 - 31st Dec 2020. It covers credit cards of 1000 customers doing transactions with a pool of 800 merchants.

### Descripción de los datos

### Métricas de evaluación

## Importando librerías

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import plotly.express as px


## Carga de datos

In [3]:
test_original = pd.read_csv('C:/Users/ramos/Documents/Tripleten/Proyectos_after_bootcamp/Deteccion_fraudes_TDC/data/fraudTest.csv')
train_original = pd.read_csv('C:/Users/ramos/Documents/Tripleten/Proyectos_after_bootcamp/Deteccion_fraudes_TDC/data/fraudTrain.csv')

# test_original = pd.read_csv('C:/Users/omarh\Documents/Tripleten/Proyectos_after_bootcamp/Deteccion_fraudes_TDC/data/fraudTest.csv')
# train_original = pd.read_csv('C:/Users/omarh\Documents/Tripleten/Proyectos_after_bootcamp/Deteccion_fraudes_TDC/data/fraudTrain.csv')


### Conociendo el contenido de los dataframes:

In [4]:
# test_original.info()
# train_original.info()

In [5]:
#De manera intuitiva podemos saber que los dataframes tienen las mismas columnas, sin embargo realizamos una verificación
# Vamos a unificar los datos para realizar el EDA general, posteriormente vamos separar los datos de entrenamiento y prueba

raw_data = pd.concat([test_original,train_original], axis=0)
#eliminamos la primer columna "Unnamed:0" que es un duplicado del índice
raw_data = raw_data.drop(columns=raw_data.columns[0],axis=1)

raw_data.head(3)
raw_data.info(show_counts=True)


<class 'pandas.core.frame.DataFrame'>
Index: 1852394 entries, 0 to 1296674
Data columns (total 22 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trans_date_trans_time  1852394 non-null  object 
 1   cc_num                 1852394 non-null  int64  
 2   merchant               1852394 non-null  object 
 3   category               1852394 non-null  object 
 4   amt                    1852394 non-null  float64
 5   first                  1852394 non-null  object 
 6   last                   1852394 non-null  object 
 7   gender                 1852394 non-null  object 
 8   street                 1852394 non-null  object 
 9   city                   1852394 non-null  object 
 10  state                  1852394 non-null  object 
 11  zip                    1852394 non-null  int64  
 12  lat                    1852394 non-null  float64
 13  long                   1852394 non-null  float64
 14  city_pop               

#### **Descripción del Dataframe**
tenemos un dataframe con 1,852,394 registros no nulos

##### Descripción de columnas
* trans_date_trans_time	 : Fecha de transacción 
    * tipo object, *_**será cambiado a datetime**_*
* ccnum : Número de transacción 
    * tipo int64
* merchant : nombe del comercio donde se hizo la transacción
    * tipo object
* category : categoría
    * tipo object
* amt : monto de la transacción
    * tipo float64
* first : primer nombre del titular de la tarjeta
    * tipo object
* last : apellido del titular de la tarjeta
     * tipo object
* gender: género del titular de la tarjeta
    * tipo object
* street : calle del titular
    * tipo object
* city : ciudad del titular
    * tipo object
* state :  estado del titular
    * tipo object
* zip : código postal del titular
    * tipo int64
* lat : latitud asociada a la dirección del titular
    * tipo float64 
* long : longitud asociada a la dirección del titular
    * tipo float64
* city_pop : población de la ciudad del titular de la tarjeta
    * tipo int64
* job : profesión del titular
    * tipo object
* dob : fecha de nacimiento del titular
    * tipo object, *_**será cambiado a datetime**_*
* trans_num : número único de transacción
    * tipo object
* unix_time : tiempo de la transacción en formato unix
    * tipo int64 , *_**Será eliminado**_*
* merch_lat : latitud de la ubicación del comerciante
    * tipo float64
* merch_long : longitud de la ubicación del comerciante
    * tipo float64
* is_fraud : es fraude 
    * tipo int64

## Preprocesamiento de datos

### Trabajando con dataset **raw_data**
* Eliminación de columnas innecesarias
* Conversión de tipo de datos en columnas
* Verificación de nulos
* Eliminación de duplicados
* Creación de dataframe con una muestra estratificada

In [6]:
raw_data.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [7]:
#Eliminando columna 'unix_time'
raw_data = raw_data.drop(columns='unix_time')


In [8]:
#convirtiendo 'trans_date_trans_time'  a datetime
raw_data['trans_date_trans_time'] = pd.to_datetime(raw_data['trans_date_trans_time'])
display(raw_data['trans_date_trans_time'].sample(2))

raw_data['dob'] = pd.to_datetime(raw_data['dob'])
display(raw_data['dob'].sample(2))


623477   2019-09-22 09:58:45
682938   2019-10-18 19:44:40
Name: trans_date_trans_time, dtype: datetime64[ns]

398199   1997-07-05
131028   1963-06-30
Name: dob, dtype: datetime64[ns]

In [9]:
#Verificación de nulos
raw_data.isnull().any()


trans_date_trans_time    False
cc_num                   False
merchant                 False
category                 False
amt                      False
first                    False
last                     False
gender                   False
street                   False
city                     False
state                    False
zip                      False
lat                      False
long                     False
city_pop                 False
job                      False
dob                      False
trans_num                False
merch_lat                False
merch_long               False
is_fraud                 False
dtype: bool

In [10]:
#Eliminación de duplicados
print('filas antes de eliminación de duplicados:',raw_data.shape[0] )
raw2=raw_data.drop_duplicates()
print('Filas después de eliminación de duplicados',raw2.shape[0])
raw_data.shape

filas antes de eliminación de duplicados: 1852394
Filas después de eliminación de duplicados 1852394


(1852394, 21)

In [11]:
#Creando una copia del dataframe para trabajar
# data = raw_data.copy()

In [38]:
#Creando un dataframe con una muestra estratificada del 50%
sampled_data,_ = train_test_split(raw_data,
                                  test_size=0.1,
                                    stratify=raw_data['is_fraud'],
                                      random_state=1234)

In [39]:
display(sampled_data.shape)

(1667154, 21)

## Análisis exploratorio de los datos (EDA)

In [14]:
sampled_data.describe()

,trans_date_trans_time,cc_num,amt,zip,lat,long,city_pop,dob,merch_lat,merch_long,is_fraud
count,1481915,1.481915e+06,1.481915e+06,1.481915e+06,1.481915e+06,1.481915e+06,1.481915e+06,1481915,1.481915e+06,1.481915e+06,1.481915e+06
mean,2020-01-20 23:06:30.731598848,4.182601e+17,7.014825e+01,4.880241e+04,3.854021e+01,-9.022375e+01,8.877803e+04,1973-10-18 20:12:14.140892080,3.853956e+01,-9.022364e+01,5.210150e-03
min,2019-01-01 00:00:18,6.041621e+10,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1924-10-30 00:00:00,1.902742e+01,-1.666701e+02,0.000000e+00
25%,2019-07-23 08:42:37.500000,1.800429e+14,9.640000e+00,2.623700e+04,3.466890e+01,-9.679800e+01,7.430000e+02,1962-08-13 00:00:00,3.474527e+01,-9.689456e+01,0.000000e+00
50%,2020-01-02 09:10:03,3.521417e+15,4.749000e+01,4.817400e+04,3.935430e+01,-8.747690e+01,2.443000e+03,1975-11-30 00:00:00,3.936998e+01,-8.743820e+01,0.000000e+00
75%,2020-07-23 12:50:57.500000,4.642255e+15,8.314000e+01,7.201100e+04,4.189480e+01,-8.015800e+01,2.032800e+04,1987-04-23 00:00:00,4.195478e+01,-8.023937e+01,0.000000e+00
max,2020-12-31 23:59:34,4.992346e+18,2.894890e+04,9.992100e+04,6.669330e+01,-6.795030e+01,2.906700e+06,2005-01-29 00:00:00,6.751027e+01,-6.695090e+01,1.000000e+00
std,NaN,1.310390e+18,1.609217e+02,2.688394e+04,5.069070e+00,1.374867e+01,3.020539e+05,NaN,5.103454e+00,1.376041e+01,7.199311e-02


In [15]:
sampled_data.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [119]:
#Top 10 empleos que sufrieron fraudes
top_jobs_fraud = pd.DataFrame(sampled_data[sampled_data['is_fraud']==1][['job','is_fraud']].value_counts().sort_values(ascending=False).head(10))
top_jobs_fraud.reset_index(inplace=True)

top_jobs_fraud = top_jobs_fraud.drop(columns='is_fraud')
px.bar(top_jobs_fraud, x='job',y='count',color='job',title='Top 10 empleos que sufrieron fraude')






In [120]:
#Explorando la cantidad de fraudes por ciudad
top_cities_fraud = pd.DataFrame(sampled_data[sampled_data['is_fraud']==1][['city','is_fraud']].value_counts().sort_values(ascending=False).head(10))
top_cities_fraud.reset_index(inplace=True)
top_cities_fraud.drop(columns='is_fraud',inplace=True)
px.bar(top_cities_fraud, x='city', y='count', color='city', title='Top 10 ciudades donde viven las víctimas de fraude')


In [154]:
sampled_data[sampled_data['merch_long']=='-84.938483']

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,zip,lat,long,city_pop,job,dob,trans_num,merch_lat,merch_long,is_fraud


In [147]:
filtered_data = sampled_data[(sampled_data['merch_lat'] == 30.630093) & (sampled_data['merch_long'] == -84.938483)]
filtered_data


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,zip,lat,long,city_pop,job,dob,trans_num,merch_lat,merch_long,is_fraud


In [142]:
#explorando ubicación del fraude
fraud_location= pd.DataFrame(sampled_data[sampled_data['is_fraud']==1][['merch_long','merch_lat','is_fraud']].value_counts().sort_values(ascending=False))
# .head(10))
fraud_location.reset_index(inplace=True)
fraud_location.drop(columns='is_fraud',inplace=True)
fraud_location.sample(10)


,merch_long,merch_lat,count
4882,-84.938483,30.630093,1
4474,-86.628922,43.431190,1
7205,-77.101597,39.142665,1
1824,-98.047707,32.061705,1
274,-122.483321,38.009217,1
4683,-85.842196,32.149248,1
4058,-88.377123,41.976120,1
5480,-82.715714,42.514009,1
1642,-98.908303,40.515299,1
4521,-86.426062,40.657616,1


* La fecha mínima que tenemos de datos es el 01/01/2019
* La fecha máxima que tenemos es 31/12/2020
* los montos de transacción van desde los 70 dls hasta los 28,948
* Tenemos usuarios nacidos desde 1924 hasta 2005 (desde 100 años hasta 19 años al día de hoy)

Podemos observar que los empleos más propensos a fraudes fueron:
* Quantity Surveyor - 64 incidencias
* Materials Engineer - 58 incidencias
* Audiological Scientist - 55 incidencias
* Naval architec - 53 incidencias
* Trading standards officer - 51 incidencias





In [15]:
# # Explorando la frecuencia de fraudes por ciudad
# fig = px.bar(sampled_data['job'])
# fig.show()

In [55]:
# sampled_data2 = sampled_data.copy()

In [54]:
# sampled_data2['amt'] = sampled_data2['amt'] +1
# sampled_data2['log_amt'] = np.log(sampled_data2['amt'])
# fig = px.histogram(sampled_data2['log_amt'])
# fig.show()

In [53]:
# sampled_data2['log_amt'].hist(bins=50)
# plt.title('Distribución Logarítmica de Montos de Transacciones')
# plt.xlabel('Logaritmo del Monto')
# plt.ylabel('Frecuencia')
# plt.show()

In [52]:
# #explorando la distribución de los montos de transacción
# fig = px.histogram(sampled_data['amt'])
# fig.show()

### Unificando y analizando dataframes

### Conclusiones EDA

## Creando dataset para el modelo

### Imputación de nulos

## Análisis y Selección de características (Feature engineering)

## Codificación y estandarización de los datos

### Categorización de datos 
(Onehot por ejemplo)
evaluar mantener una copia de datos sin escalar

### Escalando datos

## Manejo de desabalanceo de datos

## Creación del modelo y modelo benchmark